In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
# #         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import re
import os
import sys
import time
import copy
import tqdm
import random
from collections import defaultdict
import torch
import shutil
import pandas as pd
from skimage import io, transform,color
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms as T
# from albumentations.pytorch import ToTensor
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, models
from torch import nn
from functools import wraps
# from torchvision import models

from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
import cv2
from albumentations.pytorch import ToTensorV2
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from PIL import Image
from torch import nn
import zipfile

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



In [ ]:
"""
Set the path for lucchi and kasthuri dataset.
"""


train_img_path_luchi = "/kaggle/input/segmentation-dataset/lucchi_pp/Lucchi++/Train_In/"
train_mask_path_luchi = "/kaggle/input/segmentation-dataset/lucchi_pp/Lucchi++/Train_Out/"

test_img_path_luchi = "/kaggle/input/segmentation-dataset/lucchi_pp/Lucchi++/Test_In/"
test_mask_path_luchi = "/kaggle/input/segmentation-dataset/lucchi_pp/Lucchi++/Test_Out/"


train_img_path_kasthuri = "/kaggle/input/segmentation-dataset/kasthuri_pp/Kasthuri++/Train_In/"
train_mask_path_kasthuri = "/kaggle/input/segmentation-dataset/kasthuri_pp/Kasthuri++/Train_Out/"

test_img_path_kasthuri = "/kaggle/input/segmentation-dataset/kasthuri_pp/Kasthuri++/Test_In/"
test_mask_path_kasthuri = "/kaggle/input/segmentation-dataset/kasthuri_pp/Kasthuri++/Test_Out/"


glas_path = '../input/glas-dataset/Warwick QU Dataset (Released 2016_07_08)/'

In [ ]:
# def mask_convert(mask):
#     mask = mask.clone().cpu().detach().numpy()
#     mask = mask.transpose((1,2,0))
#     std = np.array((0.5))
#     mean = np.array((0.5))
#     mask  = std * mask + mean
#     mask = mask.clip(0,1)
#     mask = np.squeeze(mask)
#     return mask

# # converting tensor to image
# def image_convert(image):
#     image = image.clone().cpu().numpy()
#     image = image.transpose((1,2,0))
#     std = np.array((0.5,0.5,0.5))
#     mean = np.array((0.5,0.5,0.5))
#     image  = std * image + mean
#     image = image.clip(0,1)
#     image = (image * 255).astype(np.uint8)
#     return image

In [ ]:
"""
Function that returns a series of augmentations.
"""

import albumentations as A


def get_valid_transforms():
    return A.Compose(
        [
            A.Normalize(mean=(0),std=(1))
        ],
        p=1.0)


def light_training_transforms():
    return A.Compose([
        A.OneOf(
            [
                A.Transpose(),
                A.VerticalFlip(),
                A.HorizontalFlip(),
                A.RandomRotate90(),
                A.NoOp(),
            ], p=1.0),
    ])


def medium_training_transforms():
    return A.Compose([
        A.OneOf(
            [
                A.Transpose(),
                A.VerticalFlip(),
                A.HorizontalFlip(),
                A.RandomRotate90(),
                A.NoOp()
            ], p=1.0),
        A.OneOf(
            [
                A.CoarseDropout(max_holes=16, max_height=16, max_width=16),
                A.NoOp()
            ], p=1.0),
    ])

# 0.5517, std = 0.1187


def heavy_training_transforms():
    return A.Compose([
        A.OneOf(
            [
                A.Transpose(),
                A.VerticalFlip(),
                A.HorizontalFlip(),
                A.RandomRotate90(),
                A.NoOp(),
            ], p=1.0),
        A.OneOf(
            [
                A.CLAHE(),
#                 A.RGBShift(),
                A.RandomBrightnessContrast(),
                A.RandomGamma(),
#                 A.HueSaturationValue(),
                A.NoOp(),
            ], p=0.15),
        A.OneOf(
            [
                A.ElasticTransform(),
                A.GridDistortion(),
                A.OpticalDistortion(),
                A.NoOp(),
                A.ShiftScaleRotate(),
            ], p=1.0),
        A.OneOf(
            [
                A.GaussNoise(),
                A.GaussianBlur(),
                A.NoOp(),
            ], p=0.15),
        A.OneOf(
            [
                A.CoarseDropout(max_holes=16, max_height=16, max_width=16),
                A.NoOp(),
            ], p=1.0),
    ])

In [ ]:
"""
Funciton to pair image and its corresponding mask in a list.

"""


def preProcessImg_luchi_kasthuri(train_path,test_path,data):
  regex = "^0+(?!$)"

  train_mask = []
  train_image = []

  test_img = []
  test_mask = []

  if data == 'luchi':
    for file in os.listdir(train_path):
      files = file
      if files.endswith('png'):
        files = files.split('mask')[1]
        files = re.sub(regex, "", files)
        if files == '.png':
          files = '0.png'
        train_image.append(file)
        train_mask.append(files)

    for file in os.listdir(test_path):
      files = file
      if files.endswith('png'):
        files = files.split("mask")[1]
        files = re.sub(regex, "", files)
        if files == '.png':
          files = '0.png'
        test_img.append(file)
        test_mask.append(files)
  else:
    for file in os.listdir(train_path):
      if file.endswith('png'):
        train_image.append(file)
        train_mask.append(file)

    for file in os.listdir(test_path):
      if file.endswith('png'):
        test_img.append(file)
        test_mask.append(file)

  return train_image,train_mask,test_img,test_mask

In [ ]:
"""
The dataset class that returns the (image, mask) in its tensor form.

Dataset_Preparation takes the lucchi training images. It takes the dataframe(containing the path to image and its mask) and the augmentation function as input.
Dataset_Preparation_test takes the lucchi test images. It takes the dataframe(containing the path to image and its mask).
Dataset_Preparation_kasthuri takes the kasthuri training images. It takes the dataframe(containing the path to image and its mask) and the augmentation function as input.
Dataset_Preparation_kasthuri_test takes the kasthuri test images. It takes the dataframe(containing the path to image and its mask).

"""


class Dataset_Preparation(Dataset):
  def __init__(self,df,transform=None,Normalize_transform = None):
    self.df = df
    self.transforms = transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    train_path = train_img_path_luchi + self.df['image'][idx]
    mask_path = train_mask_path_luchi + self.df['mask'][idx]

    X  = cv2.imread(train_path, cv2.IMREAD_GRAYSCALE)
    X = np.expand_dims(X,axis=-1)
    
    Y = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
    Y = np.expand_dims(Y,axis=-1)
    
    mask = np.zeros((Y.shape[0],Y.shape[1],1))
    mask = np.maximum(mask,Y)
    mask = mask.astype("float32")
    
    if transform:
        transformed = self.transforms(image=X, mask = mask)
        img = transformed['image']
        mask = transformed['mask']
    

    
    img = cv2.resize(img,(224,224),interpolation=cv2.INTER_NEAREST)
    mask = cv2.resize(mask,(224,224),interpolation=cv2.INTER_NEAREST)    
    mask = mask/255
    img = img/255
#     img=img-0.5517
#     img=img/0.1187
    mask = np.expand_dims(mask,axis=-1).transpose((2,0,1))
    img = np.expand_dims(img,axis=0)
        
    img = torch.from_numpy(img)
    img.unsqueeze(1)
    mask = torch.from_numpy(mask)

    return (img,mask)


class Dataset_Preparation_test(Dataset):
  def __init__(self,df,transform = None):
    self.df = df
    self.transforms = transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    img_path = test_img_path_luchi + self.df['image'][idx]
    mask_path = test_mask_path_luchi + self.df['mask'][idx]

    X = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    X = np.expand_dims(X,axis=-1)

    Y = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
    Y = np.expand_dims(Y,axis=-1)

    mask = np.zeros((Y.shape[0],Y.shape[1],1))
    mask = np.maximum(mask,Y)
    mask = mask.astype("float32")
    
#     if transform:
#         transformed = self.transforms(image=X, mask = mask)
#         img = transformed['image']
#         mask = transformed['mask']
    
    img = cv2.resize(X,(224,224),interpolation=cv2.INTER_NEAREST)
    mask = cv2.resize(mask,(224,224),interpolation=cv2.INTER_NEAREST)

    mask = mask/255
    img = img/255
    mask = np.expand_dims(mask,axis=-1).transpose((2,0,1))
    img = np.expand_dims(img,axis=0)

    img = torch.from_numpy(img)
    img.unsqueeze(0)
    mask = torch.from_numpy(mask)

    return (img,mask)


class Dataset_Preparation_kasthuri(Dataset):
  def __init__(self,df,transform=None,Normalize_transform = None):
    self.df = df
    self.transforms = transform
    self.Normalize = Normalize_transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    img_path = train_img_path_kasthuri + self.df['image'][idx]
    mask_path = train_mask_path_kasthuri + self.df['mask'][idx]

    X = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    X = np.expand_dims(X,axis=-1)
    Y = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
    Y = np.where(Y==2,0,Y)
    Y = np.expand_dims(Y,axis=-1)

    mask = np.zeros((Y.shape[0],Y.shape[1],1))
    mask = np.maximum(mask,Y)
    mask = mask.astype('float32')

#     img,mask = self.transforms(X,mask)
    if transform:
        transformed = self.transforms(image=X, mask = mask)
        img = transformed['image']
        mask = transformed['mask']

    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_NEAREST)
    mask = cv2.resize(mask,(224,224),interpolation = cv2.INTER_NEAREST)

    mask = mask/255
    img = img/255

    mask = np.expand_dims(mask,axis=-1).transpose((2,0,1))
    img = np.expand_dims(img,axis=0)
    # print(img.shape)
    
    
    img = torch.from_numpy(img)
    img.unsqueeze(0)
    mask = torch.from_numpy(mask)

    return (img,mask)


class Dataset_Preparation_kasthuri_test(Dataset):
  def __init__(self,df,transform=None):
    self.df = df
    self.transforms = transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    img_path = test_img_path_kasthuri + self.df['image'][idx]
    mask_path = test_mask_path_kasthuri + self.df['mask'][idx]

#     X = io.imread(img_path)
    X = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    X = np.expand_dims(X,axis=-1)

    Y = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
    Y = np.where(Y==2,0,Y)
#     Y = color.rgb2gray(Y)
    Y = np.expand_dims(Y,axis=-1)

    mask = np.zeros((Y.shape[0],Y.shape[1],1))
    mask = np.maximum(mask,Y)
    mask = mask.astype('float32')

    # img,mask = self.transforms(X,mask)
    
#     if transform:
#         transformed = self.transforms(image=X, mask = mask)
#         img = transformed['image']
#         mask = transformed['mask']
    
    img = cv2.resize(X,(224,224),interpolation = cv2.INTER_NEAREST)
    mask = cv2.resize(mask,(224,224),interpolation = cv2.INTER_NEAREST)

    mask = mask/255
    img = img/255

    mask = np.expand_dims(mask,axis=-1).transpose((2,0,1))
    img = np.expand_dims(img,axis=0)
    # print(img.shape)

    img = torch.from_numpy(img)
    img.unsqueeze(0)
    mask = torch.from_numpy(mask)

    return (img,mask)

In [ ]:
"""
returns img and its mask as list which is later converted into a dataframe.
"""

train_img_luchi, train_mask_luchi, test_img_luchi,test_mask_luchi = preProcessImg_luchi_kasthuri(train_img_path_luchi,test_img_path_luchi,data='luchi')
train_img_kasthuri, train_mask_kasthuri, test_img_kasthuri,test_mask_kasthuri = preProcessImg_luchi_kasthuri(train_img_path_kasthuri,test_img_path_kasthuri,data = 'kasthuri')


"""
Create dataframe from the output of the "preProcessImg_luchi_kasthuri" function.
"""
df_train_luchi = pd.DataFrame({"image":train_img_luchi,"mask":train_mask_luchi})
df_test_luchi = pd.DataFrame({"image":test_img_luchi,"mask":test_mask_luchi})

df_train_kasthuri = pd.DataFrame({"image":train_img_kasthuri, "mask":train_mask_kasthuri})
df_test_kasthuri = pd.DataFrame({"image":test_img_kasthuri, "mask":test_mask_kasthuri})


"""
create a (image,mask) tensor pair object of Dataset class.
"""
data_train_luchi = Dataset_Preparation(df_train_luchi,heavy_training_transforms())
data_test_luchi = Dataset_Preparation_test(df_test_luchi)

data_train_kasthuri = Dataset_Preparation_kasthuri(df_train_kasthuri,heavy_training_transforms())
data_test_kasthuri = Dataset_Preparation_kasthuri_test(df_test_kasthuri)



"""
Divide the object of Dataset class into train and val 
"""
train_split_luchi = int(data_train_luchi.__len__()*0.8)
test_split_luchi  = int(data_train_luchi.__len__()-train_split_luchi)

train_split_kasthuri = int(data_train_kasthuri.__len__()*0.8)
test_split_kasthuri  = int(data_train_kasthuri.__len__()-train_split_kasthuri)

trainset_luchi, valset_luchi = random_split(data_train_luchi, [train_split_luchi,test_split_luchi])
trainset_kasthuri,valset_kasthuri = random_split(data_train_kasthuri,[train_split_kasthuri,test_split_kasthuri])


"""
Creates a DataLoader object for train, val and test set 
"""
train_loader_segmentation_luchi = torch.utils.data.DataLoader(dataset=trainset_luchi, batch_size=4,shuffle=True)
val_loader_segmentation_luchi = torch.utils.data.DataLoader(dataset=valset_luchi, batch_size=4,shuffle=True)
test_loader_segmentation_luchi = torch.utils.data.DataLoader(dataset=data_test_luchi,batch_size=4,shuffle=True)

train_loader_segmentation_kasthuri = torch.utils.data.DataLoader(dataset=trainset_kasthuri, batch_size=4,shuffle=True)
val_loader_segmentation_kasthuri = torch.utils.data.DataLoader(dataset=valset_kasthuri, batch_size=4,shuffle=True)
test_loader_segmentation_kasthuri = torch.utils.data.DataLoader(dataset=data_test_kasthuri,batch_size=4,shuffle=True)



In [ ]:

"""show one instance of image and its mask."""

for idx, (img,mask) in enumerate(train_loader_segmentation_kasthuri):
    
    if idx==1:
#         print("Hello")
#         print(img.shape)
#         print(mask.shape)
#         print(np.unique(mask[0].detach().cpu().numpy().flatten()))
#         print(np.unique(img[0].detach().cpu().numpy().flatten()))
        plt.subplot(1,2,1)
        plt.imshow(np.asarray(img[0].detach().cpu()).squeeze())
        plt.subplot(1,2,2)
        plt.imshow(np.asarray(mask[0].detach().cpu()).squeeze())
#     break


In [ ]:
# some utility functions
def mask_convert(mask):
    mask = mask.clone().cpu().detach().numpy()
    mask = mask.transpose((1,2,0))
    std = np.array((0.5))
    mean = np.array((0.5))
    mask  = std * mask + mean
    mask = mask.clip(0,1)
    mask = np.squeeze(mask)
    return mask

# converting tensor to image
def image_convert(image):
    image = image.clone().cpu().numpy()
    image = image.transpose((1,2,0))
    std = np.array((0.5,0.5,0.5))
    mean = np.array((0.5,0.5,0.5))
    image  = std * image + mean
    image = image.clip(0,1)
    image = (image * 255).astype(np.uint8)
    return image


In [ ]:
def plot_img(no_):
    iter_ = iter(train_loader)
    images,masks = next(iter_)
    images = images.to(device)
    masks = masks.to(device)
    plt.figure(figsize=(10,6))
    for idx in range(0,no_):
        
        image = image_convert(images[idx])
        plt.subplot(2,no_,idx+1)
        plt.title('image')
        plt.imshow(image)
    for idx in range(0,no_):
        mask = mask_convert(masks[idx])
        plt.subplot(2,no_,idx+no_+1)
        plt.title('mask')
        plt.imshow(mask,cmap='gray')
    plt.show()

In [ ]:
#stand alone self-attention module

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

import math


class AttentionConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, bias=False):
        super(AttentionConv, self).__init__()
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.groups = groups

        assert self.out_channels % self.groups == 0, "out_channels should be divided by groups. (example: out_channels: 40, groups: 4)"

        self.rel_h = nn.Parameter(torch.randn(out_channels // 2, 1, 1, kernel_size, 1), requires_grad=True)
        self.rel_w = nn.Parameter(torch.randn(out_channels // 2, 1, 1, 1, kernel_size), requires_grad=True)

        self.key_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
        self.query_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
        self.value_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)

        self.reset_parameters()

    def forward(self, x):
        batch, channels, height, width = x.size()

        padded_x = F.pad(x, [self.padding, self.padding, self.padding, self.padding])
        q_out = self.query_conv(x)
        k_out = self.key_conv(padded_x)
        v_out = self.value_conv(padded_x)

        k_out = k_out.unfold(2, self.kernel_size, self.stride).unfold(3, self.kernel_size, self.stride)
        v_out = v_out.unfold(2, self.kernel_size, self.stride).unfold(3, self.kernel_size, self.stride)

        k_out_h, k_out_w = k_out.split(self.out_channels // 2, dim=1)
        k_out = torch.cat((k_out_h + self.rel_h, k_out_w + self.rel_w), dim=1)

        k_out = k_out.contiguous().view(batch, self.groups, self.out_channels // self.groups, height, width, -1)
        v_out = v_out.contiguous().view(batch, self.groups, self.out_channels // self.groups, height, width, -1)

        q_out = q_out.view(batch, self.groups, self.out_channels // self.groups, height, width, 1)

        out = q_out * k_out
        out = F.softmax(out, dim=-1)
        out = torch.einsum('bnchwk,bnchwk -> bnchw', out, v_out).view(batch, -1, height, width)

        return out

    def reset_parameters(self):
        init.kaiming_normal_(self.key_conv.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.value_conv.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.query_conv.weight, mode='fan_out', nonlinearity='relu')

        init.normal_(self.rel_h, 0, 1)
        init.normal_(self.rel_w, 0, 1)


class AttentionStem(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, m=4, bias=False):
        super(AttentionStem, self).__init__()
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.groups = groups
        self.m = m

        assert self.out_channels % self.groups == 0, "out_channels should be divided by groups. (example: out_channels: 40, groups: 4)"

        self.emb_a = nn.Parameter(torch.randn(out_channels // groups, kernel_size), requires_grad=True)
        self.emb_b = nn.Parameter(torch.randn(out_channels // groups, kernel_size), requires_grad=True)
        self.emb_mix = nn.Parameter(torch.randn(m, out_channels // groups), requires_grad=True)

        self.key_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
        self.query_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
        self.value_conv = nn.ModuleList([nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias) for _ in range(m)])

        self.reset_parameters()

    def forward(self, x):
        batch, channels, height, width = x.size()

        padded_x = F.pad(x, [self.padding, self.padding, self.padding, self.padding])

        q_out = self.query_conv(x)
        k_out = self.key_conv(padded_x)
        v_out = torch.stack([self.value_conv[_](padded_x) for _ in range(self.m)], dim=0)

        k_out = k_out.unfold(2, self.kernel_size, self.stride).unfold(3, self.kernel_size, self.stride)
        v_out = v_out.unfold(3, self.kernel_size, self.stride).unfold(4, self.kernel_size, self.stride)

        k_out = k_out[:, :, :height, :width, :, :]
        v_out = v_out[:, :, :, :height, :width, :, :]

        emb_logit_a = torch.einsum('mc,ca->ma', self.emb_mix, self.emb_a)
        emb_logit_b = torch.einsum('mc,cb->mb', self.emb_mix, self.emb_b)
        emb = emb_logit_a.unsqueeze(2) + emb_logit_b.unsqueeze(1)
        emb = F.softmax(emb.view(self.m, -1), dim=0).view(self.m, 1, 1, 1, 1, self.kernel_size, self.kernel_size)

        v_out = emb * v_out

        k_out = k_out.contiguous().view(batch, self.groups, self.out_channels // self.groups, height, width, -1)
        v_out = v_out.contiguous().view(self.m, batch, self.groups, self.out_channels // self.groups, height, width, -1)
        v_out = torch.sum(v_out, dim=0).view(batch, self.groups, self.out_channels // self.groups, height, width, -1)

        q_out = q_out.view(batch, self.groups, self.out_channels // self.groups, height, width, 1)

        out = q_out * k_out
        out = F.softmax(out, dim=-1)
        out = torch.einsum('bnchwk,bnchwk->bnchw', out, v_out).view(batch, -1, height, width)

        return out

    def reset_parameters(self):
        init.kaiming_normal_(self.key_conv.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.query_conv.weight, mode='fan_out', nonlinearity='relu')
        for _ in self.value_conv:
            init.kaiming_normal_(_.weight, mode='fan_out', nonlinearity='relu')

        init.normal_(self.emb_a, 0, 1)
        init.normal_(self.emb_b, 0, 1)
        init.normal_(self.emb_mix, 0, 1)


In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, groups=1, base_width=64):
        super(Bottleneck, self).__init__()
        self.stride = stride
        width = int(out_channels * (base_width / 64.)) * groups

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, width, kernel_size=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            AttentionConv(width, width, kernel_size=7, padding=3, groups=8),
            nn.BatchNorm2d(width),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(width, self.expansion * out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(self.expansion * out_channels),
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        if self.stride >= 2:
            out = F.avg_pool2d(out, (self.stride, self.stride))

        out += self.shortcut(x)
        out = F.relu(out)

        return out


class Model(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1000, stem=False):
        super(Model, self).__init__()
        self.in_places = 64

        if stem:
            self.init = nn.Sequential(
                # CIFAR10
                AttentionStem(in_channels=3, out_channels=64, kernel_size=4, stride=1, padding=2, groups=1),
                nn.BatchNorm2d(64),
                nn.ReLU(),

                # For ImageNet
                # AttentionStem(in_channels=3, out_channels=64, kernel_size=4, stride=1, padding=2, groups=1),
                # nn.BatchNorm2d(64),
                # nn.ReLU(),
                # nn.MaxPool2d(4, 4)
            )
        else:
            self.init = nn.Sequential(
                # CIFAR10
                # nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),
                # nn.BatchNorm2d(64),
                # nn.ReLU(),

                # For ImageNet
                nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            )

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.dense = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_places, planes, stride))
            self.in_places = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.init(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.dense(out)

        return out


def ResNet26(num_classes=1000, stem=False):
    return Model(Bottleneck, [1, 2, 4, 1], num_classes=num_classes, stem=stem)


def ResNet38(num_classes=1000, stem=False):
    return Model(Bottleneck, [2, 3, 5, 2], num_classes=num_classes, stem=stem)


def ResNet50(num_classes=1000, stem=False):
    return Model(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, stem=stem)


def get_model_parameters(model):
    total_parameters = 0
    for layer in list(model.parameters()):
        layer_parameter = 1
        for l in list(layer.size()):
            layer_parameter *= l
        total_parameters += layer_parameter
    return total_parameters

In [ ]:
"""
This is the stand alone self-attention model
"""
model_Attention = ResNet50(num_classes=2,stem=False)

In [ ]:
"""
This is the our modified byol with colorization
"""

def default(val, def_val):
    return def_val if val is None else val

def flatten(t):
    return t.reshape(t.shape[0], -1)

def singleton(cache_key):
    def inner_fn(fn):
        @wraps(fn)
        def wrapper(self, *args, **kwargs):
            instance = getattr(self, cache_key)
            if instance is not None:
                return instance

            instance = fn(self, *args, **kwargs)
            setattr(self, cache_key, instance)
            return instance
        return wrapper
    return inner_fn

def get_module_device(module):
    return next(module.parameters()).device

def set_requires_grad(model, val):
    for p in model.parameters():
        p.requires_grad = val

def loss_fn(x, y):
    x = F.normalize(x, dim=1)
    y = F.normalize(y, dim=1)
    return 2 - 2 * (x * y).sum(dim=-1)

# augmentation utils

class RandomApply(nn.Module):
    def __init__(self, fn, p):
        super().__init__()
        self.fn = fn
        self.p = p
    def forward(self, x):
        if random.random() > self.p:
            return x
        return self.fn(x)

# exponential moving average

class EMA():
    def __init__(self, beta):
        super().__init__()
        self.beta = beta

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new

def update_moving_average(ema_updater, ma_model, current_model):
    for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):
        old_weight, up_weight = ma_params.data, current_params.data
        ma_params.data = ema_updater.update_average(old_weight, up_weight)

        
class MLP(nn.Module):
    def __init__(self, dim, projection_size, hidden_size = 4096):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, projection_size)
        )

    def forward(self, x):
        return self.net(x)


class NetWrapper(nn.Module):
    def __init__(self, net, projection_size, projection_hidden_size, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.projector = None
        self.projection_size = projection_size
        self.projection_hidden_size = projection_hidden_size

        self.hidden = {}
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, input, output):
        device = input[0].device
        self.hidden[device] = flatten(output)

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    @singleton('projector')
    def _get_projector(self, hidden):
        _, dim = hidden.shape
        projector = MLP(dim, self.projection_size, self.projection_hidden_size)
        return projector.to(hidden)
   
    def get_representation(self, x):
        if self.layer == -1:
            return self.net(x)

        if not self.hook_registered:
            self._register_hook()

        self.hidden.clear()
        _ = self.net(x)
        hidden = self.hidden[x.device]
        self.hidden.clear()

        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden

    def forward(self, x, return_projection = True):
        representation = self.get_representation(x)

        if not return_projection:
            return representation

        projector = self._get_projector(representation)
        projection = projector(representation)
        return projection, representation
class BYOL(nn.Module):
    def __init__(
        self,
        net,
        image_size,
        hidden_layer = -2,
        projection_size = 256,
        projection_hidden_size = 4096,
        augment_fn = None,
        augment_fn2 = None,
        moving_average_decay = 0.99,
        use_momentum = True
    ):
        super().__init__()
        self.net = net

        # default SimCLR augmentation

        grayscale_aug = torch.nn.Sequential(
            T.GaussianBlur((3, 3), (1.0, 2.0))
        )

        colored_aug = torch.nn.Sequential(
            RandomApply(T.ColorJitter(0.8,0.8,0.8,0.2),p=0.3),
            T.GaussianBlur((3, 3), (1.0, 2.0)),
            T.RandomHorizontalFlip()
        )

#         self.augment1 = default(augment_fn, grayscale_aug)
#         self.augment2 = default(augment_fn2,colored_aug)
        self.online_encoder = NetWrapper(net, projection_size, projection_hidden_size, layer=hidden_layer)

        self.use_momentum = use_momentum
        self.target_encoder = None
        self.target_ema_updater = EMA(moving_average_decay)

        self.online_predictor = MLP(projection_size, projection_size, projection_hidden_size)

        # get device of network and make wrapper same device
        device = get_module_device(net)
        self.to(device)

        # send a mock image tensor to instantiate singleton parameters
        self.forward(torch.randn(2, 3, image_size, image_size, device=device),torch.randn(2,3,image_size,image_size,device=device))

    @singleton('target_encoder')
    def _get_target_encoder(self):
        target_encoder = copy.deepcopy(self.online_encoder)
        set_requires_grad(target_encoder, False)
        return target_encoder

    def reset_moving_average(self):
        del self.target_encoder
        self.target_encoder = None

    def update_moving_average(self):
        assert self.use_momentum, 'you do not need to update the moving average, since you have turned off momentum for the target encoder'
        assert self.target_encoder is not None, 'target encoder has not been created yet'
        update_moving_average(self.target_ema_updater, self.target_encoder, self.online_encoder)

    def forward(
        self,
        x,y,
        return_embedding = False,
        return_projection = True
    ):
        if return_embedding:
            return self.online_encoder(x, return_projection = return_projection)

        # image_one, image_two = self.augment1(x), x
#         image_one,image_two = self.augment1(y), self.augment2(x)
        image_one, image_two = y,x


        online_proj_one, _ = self.online_encoder(image_one)
        online_proj_two, _ = self.online_encoder(image_two)

        online_pred_one = self.online_predictor(online_proj_one)
        online_pred_two = self.online_predictor(online_proj_two)

        with torch.no_grad():
            target_encoder = self._get_target_encoder() if self.use_momentum else self.online_encoder
            target_proj_one, _ = target_encoder(image_one)
            target_proj_two, _ = target_encoder(image_two)
            target_proj_one.detach_()
            target_proj_two.detach_()

        loss_one = loss_fn(online_pred_one, target_proj_two.detach())
        loss_two = loss_fn(online_pred_two, target_proj_one.detach())

        loss = loss_one + loss_two
        return loss.mean()    

In [ ]:
"""
This is the original byol
"""

import copy
import random
from functools import wraps

import torch
from torch import nn
import torch.nn.functional as F

from torchvision import transforms as T

# helper functions

def default(val, def_val):
    return def_val if val is None else val

def flatten(t):
    return t.reshape(t.shape[0], -1)

def singleton(cache_key):
    def inner_fn(fn):
        @wraps(fn)
        def wrapper(self, *args, **kwargs):
            instance = getattr(self, cache_key)
            if instance is not None:
                return instance

            instance = fn(self, *args, **kwargs)
            setattr(self, cache_key, instance)
            return instance
        return wrapper
    return inner_fn

def get_module_device(module):
    return next(module.parameters()).device

def set_requires_grad(model, val):
    for p in model.parameters():
        p.requires_grad = val

# loss fn

def loss_fn(x, y):
    x = F.normalize(x, dim=-1, p=2)
    y = F.normalize(y, dim=-1, p=2)
    return 2 - 2 * (x * y).sum(dim=-1)

# augmentation utils

class RandomApply(nn.Module):
    def __init__(self, fn, p):
        super().__init__()
        self.fn = fn
        self.p = p
    def forward(self, x):
        if random.random() > self.p:
            return x
        return self.fn(x)

# exponential moving average

class EMA():
    def __init__(self, beta):
        super().__init__()
        self.beta = beta

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new

def update_moving_average(ema_updater, ma_model, current_model):
    for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):
        old_weight, up_weight = ma_params.data, current_params.data
        ma_params.data = ema_updater.update_average(old_weight, up_weight)

# MLP class for projector and predictor

class MLP(nn.Module):
    def __init__(self, dim, projection_size, hidden_size = 4096):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, projection_size)
        )

    def forward(self, x):
        return self.net(x)

# a wrapper class for the base neural network
# will manage the interception of the hidden layer output
# and pipe it into the projecter and predictor nets

class NetWrapper(nn.Module):
    def __init__(self, net, projection_size, projection_hidden_size, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.projector = None
        self.projection_size = projection_size
        self.projection_hidden_size = projection_hidden_size

        self.hidden = {}
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, input, output):
        device = input[0].device
        self.hidden[device] = flatten(output)

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    @singleton('projector')
    def _get_projector(self, hidden):
        _, dim = hidden.shape
        projector = MLP(dim, self.projection_size, self.projection_hidden_size)
        return projector.to(hidden)

    def get_representation(self, x):
        if self.layer == -1:
            return self.net(x)

        if not self.hook_registered:
            self._register_hook()

        self.hidden.clear()
        _ = self.net(x)
        hidden = self.hidden[x.device]
        self.hidden.clear()

        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden

    def forward(self, x, return_projection = True):
        representation = self.get_representation(x)

        if not return_projection:
            return representation

        projector = self._get_projector(representation)
        projection = projector(representation)
        return projection, representation

# main class

class BYOL_original(nn.Module):
    def __init__(
        self,
        net,
        image_size,
        hidden_layer = -2,
        projection_size = 256,
        projection_hidden_size = 4096,
        augment_fn = None,
        augment_fn2 = None,
        moving_average_decay = 0.99,
        use_momentum = True
    ):
        super().__init__()
        self.net = net

        # default SimCLR augmentation

        DEFAULT_AUG = torch.nn.Sequential(
            RandomApply(
                T.ColorJitter(0.8, 0.8, 0.8, 0.2),
                p = 0.3
            ),
            T.RandomGrayscale(p=0.2),
            T.RandomHorizontalFlip(),
            RandomApply(
                T.GaussianBlur((3, 3), (1.0, 2.0)),
                p = 0.2
            ),
            T.RandomResizedCrop((image_size, image_size)),
            T.Normalize(
                mean=torch.tensor([0.485, 0.456, 0.406]),
                std=torch.tensor([0.229, 0.224, 0.225])),
        )

        self.augment1 = default(augment_fn, DEFAULT_AUG)
        self.augment2 = default(augment_fn2, self.augment1)

        self.online_encoder = NetWrapper(net, projection_size, projection_hidden_size, layer=hidden_layer)

        self.use_momentum = use_momentum
        self.target_encoder = None
        self.target_ema_updater = EMA(moving_average_decay)

        self.online_predictor = MLP(projection_size, projection_size, projection_hidden_size)

        # get device of network and make wrapper same device
        device = get_module_device(net)
        self.to(device)

        # send a mock image tensor to instantiate singleton parameters
        self.forward(torch.randn(2, 3, image_size, image_size, device=device))

    @singleton('target_encoder')
    def _get_target_encoder(self):
        target_encoder = copy.deepcopy(self.online_encoder)
        set_requires_grad(target_encoder, False)
        return target_encoder

    def reset_moving_average(self):
        del self.target_encoder
        self.target_encoder = None

    def update_moving_average(self):
        assert self.use_momentum, 'you do not need to update the moving average, since you have turned off momentum for the target encoder'
        assert self.target_encoder is not None, 'target encoder has not been created yet'
        update_moving_average(self.target_ema_updater, self.target_encoder, self.online_encoder)

    def forward(
        self,
        x,
        return_embedding = False,
        return_projection = True
    ):
        assert not (self.training and x.shape[0] == 1), 'you must have greater than 1 sample when training, due to the batchnorm in the projection layer'

        if return_embedding:
            return self.online_encoder(x, return_projection = return_projection)

        image_one, image_two = self.augment1(x), self.augment2(x)

        online_proj_one, _ = self.online_encoder(image_one)
        online_proj_two, _ = self.online_encoder(image_two)

        online_pred_one = self.online_predictor(online_proj_one)
        online_pred_two = self.online_predictor(online_proj_two)

        with torch.no_grad():
            target_encoder = self._get_target_encoder() if self.use_momentum else self.online_encoder
            target_proj_one, _ = target_encoder(image_one)
            target_proj_two, _ = target_encoder(image_two)
            target_proj_one.detach_()
            target_proj_two.detach_()

        loss_one = loss_fn(online_pred_one, target_proj_two.detach())
        loss_two = loss_fn(online_pred_two, target_proj_one.detach())

        loss = loss_one + loss_two
        return loss.mean()


model_attention = ResNet50(num_classes=2,stem=False)    
resnet= models.resnet50(pretrained=True)

#creating object of BYOL method with resnet encoder
resnet_byol_original = BYOL_original(resnet,image_size=128).to(device)

#creating object of byol method with self-attention encoder
attention_byol_original = BYOL_original(model_attention,image_size=128).to(device)

In [ ]:
# new_resnet_wts = '/kaggle/input/new-resnet-wts/new_byol_resnet_sgd_4e3_L2loss_20epoch.pt'
# new_resnet_wts = '/kaggle/input/segmentation-dataset/new_byol_resnet_sgd_4e3_L2loss_50epoch.pt'

"""
set path for the pre-trained weights obtained after the pre-training
"""

#weights of resnet50 encoder pre-trained with our method
new_resnet_wts = '/kaggle/input/new-byol-100epochs/new_byol_resnet_sgd_4e3_L2loss_100epoch.pt'

#weights of self-attention encoder pre-trained with our method
attention_wts = '../input/new-byol-attention-100epochs/new_byol_Attention_adam_4e3_L2loss_100epoch.pt'

#weights of resnet50 encoder pre-trained with original byol
old_byol_resnet = "../input/old-byol-resnet/old_byol_resnet_adam_4e3_L2loss_50epoch.pt"

#weights of self-attention encoder pre-trained with original byol 
old_byol_attention = '../input/original-byol-attention-50epochs/old_byol_attention_adam_4e3_L2loss_50epoch.pt'

#creating object of resnet50 encoder with Imagenet weights
resnet = models.resnet50(pretrained=True)

#creating object of our method with resnet encoder
new_byol_resnet = BYOL(resnet, image_size = 128, hidden_layer = 'avgpool').to(device)

#loading the pre-trained weights in the object created above
chkpoint = torch.load(new_resnet_wts)
new_byol_resnet.load_state_dict(chkpoint['model_state_dict'])
new_byol_resnet = new_byol_resnet.to(device)


#creating an object of our method with self-attention encoder and loading the pre-trained weights
new_byol_attention = BYOL(model_Attention,image_size = 128).to(device)
chkpoint_attention = torch.load(attention_wts)
new_byol_attention.load_state_dict(chkpoint_attention['model_state_dict'])
new_byol_attention = new_byol_attention.to(device)


#loading pre-trained weights into the self-attention byol architecture
chkpoint_original_attention = torch.load(old_byol_attention)
attention_byol_original.load_state_dict(chkpoint_original_attention['model_state_dict'])
attention_byol_original = attention_byol_original.to(device)

#loading pre-trained weights into the resnet50 byol architecture
chkpoint_original_resnet = torch.load(old_byol_resnet)
resnet_byol_original.load_state_dict(chkpoint_original_resnet['model_state_dict'])
resnet_byol_original = resnet_byol_original.to(device)


In [ ]:
"""
Here, the online encoder from the objects of the byol architecture is extracted.
"""

new_encoder_Resnet = new_byol_resnet.online_encoder
new_encoder_Resnet = torch.nn.Sequential(*list(new_encoder_Resnet.children())[:-1])

new_encoder_attention = new_byol_attention.online_encoder
new_encoder_attention = torch.nn.Sequential(*list(new_encoder_attention.children())[:-1])

# old_encoder_Resnet = resnet_byol_original.online_encoder
# old_encoder_Resnet = torch.nn.Sequential(*list(old_encoder_Resnet.children())[:-1])

attention_byol_original_encoder = attention_byol_original.online_encoder
attention_byol_original_encoder = torch.nn.Sequential(*list(attention_byol_original_encoder.children())[:-1])

In [ ]:
"""
This part achieves the task of freezing the encoder extracted in the last cell
"""

# for param in new_encoder_Resnet.parameters():
#     param.requires_grad = False
    
# for param in new_encoder_attention.parameters():
#     param.requires_grad = False
    
# for param in old_encoder_Resnet.parameters():
#     param.requires_grad = False
    
# for param in attention_byol_original_encoder.parameters():
#     param.requires_grad = False

In [ ]:
#U-net architecture

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)
    
    
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

#     Unet_original = UNet(1,1)

In [ ]:
class ConvBlock(nn.Module):
    """
    Helper module that consists of a Conv -> BN -> ReLU
    """

    def __init__(self, in_channels, out_channels, padding=1, kernel_size=3, stride=1, with_nonlinearity=True):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, padding=padding, kernel_size=kernel_size, stride=stride)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.with_nonlinearity = with_nonlinearity

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if self.with_nonlinearity:
            x = self.relu(x)
        return x


class Bridge(nn.Module):
    """
    This is the middle layer of the UNet which just consists of some
    """

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.bridge = nn.Sequential(
            ConvBlock(in_channels, out_channels),
            ConvBlock(out_channels, out_channels)
        )

    def forward(self, x):
        return self.bridge(x)

class UpBlockForUNetWithResNet50Color(nn.Module):
    """
    Up block that encapsulates one up-sampling step which consists of Upsample -> ConvBlock -> ConvBlock
    """

    def __init__(self, in_channels, out_channels, up_conv_in_channels=None, up_conv_out_channels=None,
                 upsampling_method="conv_transpose"):
        super().__init__()

        if up_conv_in_channels == None:
            up_conv_in_channels = in_channels
        if up_conv_out_channels == None:
            up_conv_out_channels = out_channels

        if upsampling_method == "conv_transpose":
            self.upsample = nn.ConvTranspose2d(up_conv_in_channels, up_conv_out_channels, kernel_size=2, stride=2)
        elif upsampling_method == "bilinear":
            self.upsample = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
            )
        self.conv_block_1 = ConvBlock(in_channels, out_channels)
        self.conv_block_2 = ConvBlock(out_channels, out_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, up_x, down_x):
        """
        :param up_x: this is the output from the previous up block
        :param down_x: this is the output from the down block
        :return: upsampled feature map
        """
        x = self.upsample(up_x)
        # x = torch.cat([x, down_x], 1)
        # x = self.conv_block_1(x)
        # x = self.conv_block_2(x)
        return x
    
"""
This is the Upsampling class for resnet50 encoder
"""    
class UpBlockForUNetWithResNet50(nn.Module):
    """
    Up block that encapsulates one up-sampling step which consists of Upsample -> ConvBlock -> ConvBlock
    """

    def __init__(self, in_channels, out_channels, up_conv_in_channels=None, up_conv_out_channels=None,
                 upsampling_method="conv_transpose"):
        super().__init__()

        if up_conv_in_channels == None:
            up_conv_in_channels = in_channels
        if up_conv_out_channels == None:
            up_conv_out_channels = out_channels

        if upsampling_method == "conv_transpose":
            self.upsample = nn.ConvTranspose2d(up_conv_in_channels, up_conv_out_channels, kernel_size=2, stride=2)
        elif upsampling_method == "bilinear":
            self.upsample = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
            )
        self.conv_block_1 = ConvBlock(in_channels, out_channels)
        self.conv_block_2 = ConvBlock(out_channels, out_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, up_x, down_x):
        """
        :param up_x: this is the output from the previous up block
        :param down_x: this is the output from the down block
        :return: upsampled feature map
        """
        x = self.upsample(up_x)
        x = torch.cat([x, down_x], 1)
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        return x

"""
This is the Unet class for self-attention encoder
"""
class UNetWithAttentionEncoder(nn.Module):
    Depth = 6
    
    def __init__(self,byol_model,n_classes=2):
        super().__init__()
        down_blocks = []
        up_blocks = []
        # for child in model_Attention.children():
        #   if child.init:
        #     conv1 = child.init[0]
        #     bn1 = child.init[1]
        #     relu1 = child.init[2]
        #     self.input_block = nn.Sequential(*[conv1,bn1,relu1])
        #     self.input_pool = child.init[3]

        #   if child.layer1:
        #     down_blocks.append(child.layer1)

        #   if child.layer2:
        #     down_blocks.append(child.layer2)

        #   if child.layer3:
        #     down_blocks.append(child.layer3)

        #   if child.layer4:
        #     down_blocks.append(child.layer4)

        self.input_block = nn.Sequential(*list(byol_model.children())[0][:3])
        self.input_pool = list(byol_model.children())[0][3]

        for bottleneck in list(byol_model.children())[1:]:
            if isinstance(bottleneck, nn.Sequential):
                down_blocks.append(bottleneck)

        self.down_blocks = nn.ModuleList(down_blocks)
        self.bridge = Bridge(2048, 2048)
        up_blocks.append(UpBlockForUNetWithResNet50(2048, 1024))
        up_blocks.append(UpBlockForUNetWithResNet50(1024, 512))
        up_blocks.append(UpBlockForUNetWithResNet50(512, 256))
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=128 + 64, out_channels=128,
                                                    up_conv_in_channels=256, up_conv_out_channels=128))
        
        """
        Here , we change the value of in_channel in the Upblock as per the input. If it is a grayscale image in_channel=64+1 and if it is colored image then in_channel = 64+1 
        """
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=64 + 3, out_channels=64,
                                                    up_conv_in_channels=128, up_conv_out_channels=64))

        self.up_blocks = nn.ModuleList(up_blocks)

        self.out = nn.Conv2d(64, 1, kernel_size=1, stride=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, with_output_feature_map=False):
        pre_pools = dict()
        pre_pools[f"layer_0"] = x
        x = self.input_block(x)
        pre_pools[f"layer_1"] = x
        x = self.input_pool(x)

        for i, block in enumerate(self.down_blocks, 2):
            
          # print(block)
            x = block(x)
            if i == (UNetWithResnet50Encoder.DEPTH - 1):
                continue
            pre_pools[f"layer_{i}"] = x

        x = self.bridge(x)

        for i, block in enumerate(self.up_blocks, 1):
            key = f"layer_{UNetWithResnet50Encoder.DEPTH - 1 - i}"
            x = block(x, pre_pools[key])
        output_feature_map = x
        x = self.out(x)
        del pre_pools
        if with_output_feature_map:
            return x, output_feature_map
        else:
            return self.sigmoid(x)

"""
This is the Unet class for self-attention encoder
"""
class UNetWithResnet50Encoder(nn.Module):
    DEPTH = 6

    def __init__(self, resnet, n_classes=2):
        super().__init__()
        down_blocks = []
        up_blocks = []
        # resnet = torchvision.models.resnet.resnet50(pretrained=True)

        self.input_block = nn.Sequential(*list(resnet.children()))[:3]
        self.input_pool = list(resnet.children())[3]

        for bottleneck in list(resnet.children()):
            if isinstance(bottleneck, nn.Sequential):
                down_blocks.append(bottleneck)
                  
        self.down_blocks = nn.ModuleList(down_blocks)
        self.bridge = Bridge(2048, 2048)
        up_blocks.append(UpBlockForUNetWithResNet50(2048, 1024))
        up_blocks.append(UpBlockForUNetWithResNet50(1024, 512))
        up_blocks.append(UpBlockForUNetWithResNet50(512, 256))
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=128 + 64, out_channels=128,
                                                    up_conv_in_channels=256, up_conv_out_channels=128))
        
        """
        Here , we change the value of in_channel in the Upblock as per the input. If it is a grayscale image in_channel=64+1 and if it is colored image then in_channel = 64+1 
        """
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=64 + 3, out_channels=64,
                                                    up_conv_in_channels=128, up_conv_out_channels=64))

        self.up_blocks = nn.ModuleList(up_blocks)

        self.out = nn.Conv2d(64, 1, kernel_size=1, stride=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, with_output_feature_map=False):
        pre_pools = dict()
        pre_pools[f"layer_0"] = x
        x = self.input_block(x)
        pre_pools[f"layer_1"] = x
        x = self.input_pool(x)

        for i, block in enumerate(self.down_blocks, 2):
            # print(block)
            x = block(x)
            if i == (UNetWithResnet50Encoder.DEPTH - 1):
                continue
            pre_pools[f"layer_{i}"] = x

        x = self.bridge(x)

        for i, block in enumerate(self.up_blocks, 1):
            key = f"layer_{UNetWithResnet50Encoder.DEPTH - 1 - i}"
            x = block(x, pre_pools[key])
        output_feature_map = x
        x = self.out(x)
        del pre_pools
        if with_output_feature_map:
            return x, output_feature_map
        else:
            return self.sigmoid(x)

        
"""
This is the Unet class for self-attention encoder obtained from the BYOL(and also our method) architecture
"""        
class UNetByolWithAttentionEncoder(nn.Module):
  Depth = 6

  def __init__(self,model_for_unet_attention,n_classes=2):
    super().__init__()
    down_blocks = []
    up_blocks = []
    for child in model_for_unet_attention.children():
      if child.init:
        conv1 = child.init[0]
        bn1 = child.init[1]
        relu1 = child.init[2]
        self.input_block = nn.Sequential(*[conv1,bn1,relu1])
        self.input_pool = child.init[3]

      if child.layer1:
        down_blocks.append(child.layer1)

      if child.layer2:
        down_blocks.append(child.layer2)

      if child.layer3:
        down_blocks.append(child.layer3)

      if child.layer4:
        down_blocks.append(child.layer4)

    self.down_blocks = nn.ModuleList(down_blocks)
    self.bridge = Bridge(2048, 2048)
    up_blocks.append(UpBlockForUNetWithResNet50(2048, 1024))
    up_blocks.append(UpBlockForUNetWithResNet50(1024, 512))
    up_blocks.append(UpBlockForUNetWithResNet50(512, 256))
    up_blocks.append(UpBlockForUNetWithResNet50(in_channels=128 + 64, out_channels=128,
                                                    up_conv_in_channels=256, up_conv_out_channels=128))
    
            """
    Here , we change the value of in_channel in the Upblock as per the input. If it is a grayscale image in_channel=64+1 and if it is colored image then in_channel = 64+1 
        """
    up_blocks.append(UpBlockForUNetWithResNet50(in_channels=64 + 3, out_channels=64,
                                                    up_conv_in_channels=128, up_conv_out_channels=64))

    self.up_blocks = nn.ModuleList(up_blocks)

    self.out = nn.Conv2d(64, 1, kernel_size=1, stride=1)
    self.sigmoid = nn.Sigmoid()    
    
  def forward(self, x, with_output_feature_map=False):
    pre_pools = dict()
    pre_pools[f"layer_0"] = x
    x = self.input_block(x)
    pre_pools[f"layer_1"] = x
    x = self.input_pool(x)

    for i, block in enumerate(self.down_blocks, 2):
      x = block(x)
      if i == (UNetWithResnet50Encoder.DEPTH - 1):
        continue
      pre_pools[f"layer_{i}"] = x

    x = self.bridge(x)

    for i, block in enumerate(self.up_blocks, 1):
        key = f"layer_{UNetWithResnet50Encoder.DEPTH - 1 - i}"
        x = block(x, pre_pools[key])
    output_feature_map = x
    x = self.out(x)
    del pre_pools
    if with_output_feature_map:
      return x, output_feature_map
    else:
      return self.sigmoid(x)


"""
This is the Unet class for resnet50 encoder obtained from the BYOL(and also our method) architecture
"""
class UNetByolWithResnet50Encoder(nn.Module):
    DEPTH = 6

    def __init__(self,model_for_unet,n_classes=2):
        super().__init__()
        # resnet = torchvision.models.resnet.resnet50(pretrained=True)
        down_blocks = []
        up_blocks = []
        for child in model_for_unet.children():
          if child.conv1:
            conv1 = child.conv1
          if child.bn1:
            bn1 = child.bn1
          if child.relu:
            relu = child.relu
          self.input_block = nn.Sequential(*[conv1,bn1,relu])
        # self.input_block = nn.Sequential(*list(resnet.children()))[:3]
          # self.input_pool = list(resnet.children())[3]
          if child.maxpool:
            self.input_pool = child.maxpool

          if child.layer1:
            down_blocks.append(child.layer1)

          if child.layer2:
            down_blocks.append(child.layer2)

          if child.layer3:
            down_blocks.append(child.layer3)

          if child.layer4:
            down_blocks.append(child.layer4)
        # for bottleneck in list(resnet.children()):
        #     if isinstance(bottleneck, nn.Sequential):
        #         down_blocks.append(bottleneck)
                  
        self.down_blocks = nn.ModuleList(down_blocks)
        self.bridge = Bridge(2048, 2048)
        up_blocks.append(UpBlockForUNetWithResNet50(2048, 1024))
        up_blocks.append(UpBlockForUNetWithResNet50(1024, 512))
        up_blocks.append(UpBlockForUNetWithResNet50(512, 256))
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=128 + 64, out_channels=128,
                                                    up_conv_in_channels=256, up_conv_out_channels=128))
        
        """
        Here , we change the value of in_channel in the Upblock as per the input. If it is a grayscale image in_channel=64+1 and if it is colored image then in_channel = 64+1 
        """
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=64 + 3, out_channels=64,
                                                    up_conv_in_channels=128, up_conv_out_channels=64))

        self.up_blocks = nn.ModuleList(up_blocks)

        self.out = nn.Conv2d(64, 1, kernel_size=1, stride=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, with_output_feature_map=False):
        pre_pools = dict()
        pre_pools[f"layer_0"] = x
        x = self.input_block(x)
        pre_pools[f"layer_1"] = x
        x = self.input_pool(x)

        for i, block in enumerate(self.down_blocks, 2):
            x = block(x)
            if i == (UNetWithResnet50Encoder.DEPTH - 1):
                continue
            pre_pools[f"layer_{i}"] = x

        x = self.bridge(x)

        for i, block in enumerate(self.up_blocks, 1):
            key = f"layer_{UNetWithResnet50Encoder.DEPTH - 1 - i}"
            x = block(x, pre_pools[key])
        output_feature_map = x
        x = self.out(x)
        del pre_pools
        if with_output_feature_map:
            return x, output_feature_map
        else:
            return self.sigmoid(x)
        
        
class UNetByolWithResnet50EncoderColor(nn.Module):
    DEPTH = 6

    def __init__(self,model_for_unet,n_classes=2):
        super().__init__()
        # resnet = torchvision.models.resnet.resnet50(pretrained=True)
        down_blocks = []
        up_blocks = []
        for child in model_for_unet.children():
          if child.conv1:
            conv1 = child.conv1
          if child.bn1:
            bn1 = child.bn1
          if child.relu:
            relu = child.relu
          self.input_block = nn.Sequential(*[conv1,bn1,relu])
        # self.input_block = nn.Sequential(*list(resnet.children()))[:3]
          # self.input_pool = list(resnet.children())[3]
          if child.maxpool:
            self.input_pool = child.maxpool

          if child.layer1:
            down_blocks.append(child.layer1)

          if child.layer2:
            down_blocks.append(child.layer2)

          if child.layer3:
            down_blocks.append(child.layer3)

          if child.layer4:
            down_blocks.append(child.layer4)
        # for bottleneck in list(resnet.children()):
        #     if isinstance(bottleneck, nn.Sequential):
        #         down_blocks.append(bottleneck)
                  
        self.down_blocks = nn.ModuleList(down_blocks)
        self.bridge = Bridge(2048, 2048)
        up_blocks.append(UpBlockForUNetWithResNet50Color(2048, 1024))
        up_blocks.append(UpBlockForUNetWithResNet50Color(1024, 512))
        up_blocks.append(UpBlockForUNetWithResNet50Color(512, 256))
        up_blocks.append(UpBlockForUNetWithResNet50Color(in_channels=128 + 64, out_channels=128,
                                                    up_conv_in_channels=256, up_conv_out_channels=128))
        up_blocks.append(UpBlockForUNetWithResNet50Color(in_channels=64 + 1, out_channels=64,
                                                    up_conv_in_channels=128, up_conv_out_channels=64))

        self.up_blocks = nn.ModuleList(up_blocks)

        self.out = nn.Conv2d(64, 3, kernel_size=1, stride=1)

    def forward(self, x, with_output_feature_map=False):
        pre_pools = dict()
        pre_pools[f"layer_0"] = x
        x = self.input_block(x)
        pre_pools[f"layer_1"] = x
        x = self.input_pool(x)

        for i, block in enumerate(self.down_blocks, 2):
            x = block(x)
            if i == (UNetWithResnet50Encoder.DEPTH - 1):
                continue
            pre_pools[f"layer_{i}"] = x

        x = self.bridge(x)

        for i, block in enumerate(self.up_blocks, 1):
            key = f"layer_{UNetWithResnet50Encoder.DEPTH - 1 - i}"
            x = block(x, pre_pools[key])
        output_feature_map = x
        x = self.out(x)
        del pre_pools
        if with_output_feature_map:
            return x, output_feature_map
        else:
            return x

In [ ]:

"""
creating a object for resnet50 and setting its input channel to 1 (for grayscale image)
"""
resnet_pretrained = models.resnet50(pretrained=True)
resnet_pretrained.conv1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,stride=2,padding=3,bias = False)

# resnet_init = models.resnet50(pretrained=False)
# resnet_init.conv1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,stride=2,padding=3,bias =False)

"""
creating object for U-net with resnet50 encoder (Imagenet)
"""
resnet_preTrained_UNet = UNetWithResnet50Encoder(resnet_pretrained).to(device)
# resnet_init_UNet = UNetWithResnet50Encoder(resnet_init).to(device)

# attention_init_UNet = UNetWithAttentionEncoder().to(device)
"""
creating an object for U-net with resnet50 encoder BYOL pre-trained (Our method)
"""
new_resnet_byol_unet = UNetByolWithResnet50Encoder(new_encoder_Resnet).to(device)

"""
creating an object for U-net with resnet50 encoder BYOL pre-trained (Original)
"""
# original_resnet_byol_unet = UNetByolWithResnet50Encoder(old_encoder_Resnet).to(device)
# new_resnet_byol_unet_color = UNetByolWithResnet50EncoderColor(new_encoder_Resnet).to(device)

"""
creating an object for U-net with self-attention encoder BYOL pre-trained (our method)
"""
new_attention_byol_unet = UNetByolWithAttentionEncoder(new_encoder_attention).to(device)

"""creating an object for U-net with self-attention encoder BYOL pre-trained (our method)"""
original_attention_byol_unet = UNetByolWithAttentionEncoder(attention_byol_original_encoder).to(device)

# old_resnet_byol = UNetByolWithResnet50Encoder(old_encoder_Resnet).to(device)

In [ ]:
"""
changing the value of input_channel to 1 or 3 as per the input image.
"""

new_resnet_byol_unet.input_block[0] = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3,bias=False)
new_resnet_byol_unet = new_resnet_byol_unet.to(device)

new_attention_byol_unet.input_block[0] = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3,bias=False)
new_attention_byol_unet = new_attention_byol_unet.to(device)

# old_resnet_byol.input_block[0] = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,stride=2,padding=3,bias=False)
# old_resnet_byol = old_resnet_byol.to(device)

original_attention_byol_unet.input_block[0] = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,stride=2,padding=3,bias=False)
original_attention_byol_unet = original_attention_byol_unet.to(device)

In [ ]:
ALPHA = 0.8
GAMMA = 2

class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #first compute binary cross-entropy 
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                       
        return focal_loss


class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
#         inputs = torch.sigmoid(inputs)

        bce_weight = 0.5
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        loss_final = BCE * bce_weight + dice_loss * (1 - bce_weight)
        return loss_final
    
## IOU computation
def iou_(y_pred,y):
    inputs = y_pred.reshape(-1)
    targets = y.reshape(-1)
    intersection = (inputs * targets).sum()
    total = (inputs + targets).sum()
    union = total - intersection 
    smooth = 1    
    iou = (intersection + smooth)/(union + smooth)
    return iou



def iou_batch(y_pred,y):
    '''computes mean iou for a batch of ground truth masks and predicted masks'''
    ious = []
#     y_pred = torch.sigmoid(y_pred)
    y_pred = y_pred.clone().cpu().detach().numpy()
    y = y.clone().cpu().detach().numpy() 
    
    for pred, label in zip(y_pred, y):
        ious.append(iou_(pred, label))
    iou = np.nanmean(ious)
    return iou 


# def mIoU(pred_mask, mask, smooth=1e-10, n_classes=2):
#     with torch.no_grad():
#         pred_mask = F.softmax(pred_mask, dim=1)
        
#         pred_mask = torch.argmax(pred_mask, dim=1)
#         pred_mask = pred_mask.contiguous().view(-1)
#         mask = mask.contiguous().view(-1)
 
#         # print(pred_mask.shape)
#         # print(mask.shape)
#         iou_per_class = []
#         for clas in range(0, n_classes): #loop per pixel class
#             true_class = pred_mask == clas
#             true_label = mask == clas

#             if true_label.long().sum().item() == 0: #no exist label in this loop
#                 iou_per_class.append(np.nan)
#             else:
#                 intersect = torch.logical_and(true_class, true_label).sum().float().item()
#                 union = torch.logical_or(true_class, true_label).sum().float().item()

#                 iou = (intersect + smooth) / (union +smooth)
#                 iou_per_class.append(iou)
#         print(iou_per_class)
#         return np.nanmean(iou_per_class)
    
# def jaccard_index_numpy(y_true, y_pred):
#     """Define Jaccard index.
#        Parameters
#        ----------
#        y_true : N dim Numpy array
#            Ground truth masks. E.g. ``(num_of_images, x, y, channels)`` for 2D 
#            images or ``(volume_number, z, x, y, channels)`` for 3D volumes.
#        y_pred : N dim Numpy array
#            Predicted masks. E.g. ``(num_of_images, x, y, channels)`` for 2D 
#            images or ``(volume_number, z, x, y, channels)`` for 3D volumes.
#        Returns
#        -------
#        jac : float
#            Jaccard index value.
#     """

#     TP = np.count_nonzero(y_pred * y_true)
#     FP = np.count_nonzero(y_pred * (y_true - 1))
#     FN = np.count_nonzero((y_pred - 1) * y_true)

#     if (TP + FP + FN) == 0:
#         jac = 0
#     else:
#         jac = TP / (TP + FP + FN)

#     return jac


# def jaccard_index_numpy_without_background(y_true, y_pred):
#     """Define Jaccard index excluding the background class (first channel). 
#        Parameters
#        ----------
#        y_true : N dim Numpy array
#            Ground truth masks. E.g. ``(num_of_images, x, y, channels)`` for 2D
#            images or ``(volume_number, z, x, y, channels)`` for 3D volumes.
#        y_pred : N dim Numpy array
#            Predicted masks. E.g. ``(num_of_images, x, y, channels)`` for 2D
#            images or ``(volume_number, z, x, y, channels)`` for 3D volumes.
#        Returns
#        -------
#        jac : float
#            Jaccard index value.
#     """

#     TP = np.count_nonzero(y_pred[...,1:] * y_true[...,1:])
#     FP = np.count_nonzero(y_pred[...,1:] * (y_true[...,1:] - 1))
#     FN = np.count_nonzero((y_pred[...,1:] - 1) * y_true[...,1:])

#     if (TP + FP + FN) == 0:
#         jac = 0
#     else:
#         jac = TP / (TP + FP + FN)

#     return jac

In [ ]:
#ref https://towardsdatascience.com/how-to-save-and-load-a-model-in-pytorch-with-a-complete-example-c2920e617dee

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, checkpoint_path)
    # if it is a best model, min validation loss
    if is_best:
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(checkpoint_path, best_model_path)
        
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

In [ ]:
checkpoint_path = '/kaggle/working/chkpoint_'
best_model_path = '/kaggle/working/bestmodel.pt'
# new_resnet_byol_unet,_,_,_ = load_ckp(best_model_path, new_resnet_byol_unet,optimizer)
epochs = 50
criterion = DiceBCELoss()
#criterion = FocalLoss()
# criterion = nn.BCELoss()
learning_rate = 0.003

#set Adam optimizer and change the name of the unet as required
optimizer = torch.optim.Adam(new_attention_byol_unet.parameters(), lr=learning_rate)
# optimizer = torch.optim.RMSprop(new_attention_byol_unet.parameters(), lr=0.0005, alpha=0.99, eps=1e-08, weight_decay=0.0003, momentum=0.99, centered=False)
# optimizer = torch.optim.SGD(new_attention_byol_unet.parameters(),momentum=0.99,lr = learning_rate)

#set the scheduler
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 15, verbose = 1)
# lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,base_lr=0.000007,max_lr=0.05,step_size_up=5,mode='triangular',cycle_momentum=False)
valid_loss_min = 3.95275


"""
training of the model
"""
train_loss,val_loss = [],[]
train_iou,val_iou = [],[]

for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch + 1, epochs))
    start_time = time.time()
    
    running_train_loss = []
    running_train_score = []
    
    #change the name of the unet as required
    new_attention_byol_unet.train()
    
    #change the name of the dataloader as required
    for image,mask in train_loader_segmentation_luchi:
            image = image.to(device,dtype=torch.float)
            mask = mask.to(device,dtype=torch.float)
#             print(image.shape)

            #change the name of the unet as required
            pred_mask = new_attention_byol_unet.forward(image) 
#             print(pred_mask.shape)
#             print(mask.shape)
            # forward propogation
            loss = criterion(pred_mask,mask)
            score = iou_batch(pred_mask,mask)
            optimizer.zero_grad() # setting gradient to zero
            loss.backward()
            optimizer.step()
            running_train_loss.append(loss.item())
            running_train_score.append(score.item())
#     lr_scheduler.step(np.mean(running_train_loss))   
#     lr_scheduler.step()

              
    running_val_loss = []
    running_val_score = []
    
    """
    Validation part 
    """
    
    #change the name of the unet as required
    new_attention_byol_unet.eval()
    with torch.no_grad():
        
        for image,mask in val_loader_segmentation_luchi:
            image = image.to(device,dtype=torch.float)
            mask = mask.to(device,dtype=torch.float)
            
            #change the name of the unet as required
            pred_mask = new_attention_byol_unet.forward(image)
            loss = criterion(pred_mask,mask)
            score = iou_batch(pred_mask,mask)
            running_val_loss.append(loss.item())
            running_val_score.append(score.item())
    lr_scheduler.step(np.mean(running_val_loss)) 

    epoch_train_loss,epoch_train_score = np.mean(running_train_loss) ,np.mean(running_train_score)
    print('Train loss : {} iou : {}'.format(epoch_train_loss,epoch_train_score))                       
    train_loss.append(epoch_train_loss)
    train_iou.append(epoch_train_score)
    
    epoch_val_loss,epoch_val_score = np.mean(running_val_loss),np.mean(running_val_score)
    print('Validation loss : {} iou : {}'.format(epoch_val_loss,epoch_val_score))                                
    val_loss.append(epoch_val_loss)
    val_iou.append(epoch_val_score)
   

"""
creating checkpoint and saving model
"""
#     create checkpoint variable and add important data
    checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': epoch_val_loss,
            'state_dict': new_attention_byol_unet.state_dict(),  #change the name of the unet as required
            'optimizer': optimizer.state_dict(),
        }
    
    # save checkpoint
    save_ckp(checkpoint, False, checkpoint_path, best_model_path)
    ## TODO: save the model if validation loss has decreased
    if epoch_val_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
            # save checkpoint as best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = epoch_val_loss
    time_elapsed = time.time() - start_time
    print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [ ]:
def display_images(images, titles=None, cols=4, cmap=None, norm=None,
                   interpolation=None):
    """Display the given set of images, optionally with titles.
    images: list or array of image tensors in HWC format.
    titles: optional. A list of titles to display with each image.
    cols: number of images per row
    cmap: Optional. Color map to use. For example, "Blues".
    norm: Optional. A Normalize instance to map values to colors.
    interpolation: Optional. Image interporlation to use for display.
    """
    titles = titles if titles is not None else [""] * len(images)
    rows = len(images) // cols + 1
    plt.figure(figsize=(14, 14 * rows // cols))
    i = 1
    for image, title in zip(images, titles):
        plt.subplot(rows, cols, i)
        plt.title(title, fontsize=9)
        plt.axis('off')
        plt.imshow(image.astype(np.uint8), cmap=cmap,
                   norm=norm, interpolation=interpolation)
        i += 1
    plt.show()

In [ ]:
def plot_img(image,mask,prediction):

  plt.figure(figsize=(20,10))  
  img = image[0].squeeze()
  msk = mask[0].squeeze()
  pdt_msk = prediction[0].squeeze()

  
  pyplot.subplot(1, 3, 1) # second plot
  pyplot.imshow(img.detach().cpu())

  pyplot.subplot(1,3,2)
  pyplot.imshow(msk.detach().cpu(),cmap = 'gray')

  pyplot.subplot(1, 3, 3) # second plot
  pyplot.imshow(pdt_msk.detach().cpu(),cmap ='gray')

  
  # msk = mask[1].squeeze()
  # pdt_msk = prediction[1].squeeze()

  # pyplot.subplot(2,2,1)
  # pyplot.imshow(msk.detach().cpu())

  # pyplot.subplot(2, 2, 2) # second plot
  # pyplot.imshow(pdt_msk.detach().cpu())

  # msk = mask[2].squeeze()
  # pdt_msk = prediction[2].squeeze()

  # pyplot.subplot(3,2,1)
  # pyplot.imshow(msk.detach().cpu())

  # pyplot.subplot(3, 2, 2) # second plot
  # pyplot.imshow(pdt_msk.detach().cpu())

  # msk = mask[3].squeeze()
  # pdt_msk = prediction[3].squeeze()

  # pyplot.subplot(4,2,1)
  # pyplot.imshow(msk.detach().cpu())

  # pyplot.subplot(4, 2, 2) # second plot
  # pyplot.imshow(pdt_msk.detach().cpu())

  # msk = mask[4].squeeze()
  # pdt_msk = prediction[4].squeeze()

  # pyplot.subplot(5,2,1)
  # pyplot.imshow(msk.detach().cpu())

  # pyplot.subplot(5, 2, 2) # second plot
  # pyplot.imshow(pdt_msk.detach().cpu())

In [ ]:
"""
Function to test the model
"""

from matplotlib import pyplot
def test_model(model,dataset,criterion):

  running_test_loss = []
  running_test_score = []
  model.eval()  
  with torch.no_grad():
    for image,mask in dataset:
        image = image.to(device,dtype = torch.float)
        mask = mask.to(device,dtype = torch.float)   
        predicted_mask = model.forward(image)
        loss = criterion(predicted_mask,mask)
        score = iou_batch(predicted_mask,mask)
#         plot_img(image,mask,prediction=predicted_mask)
        running_test_loss.append(loss.item())
        running_test_score.append(score)
    return np.mean(running_test_loss),np.mean(running_test_score)

In [ ]:
# criterion = nn.BCEWithLogitsLoss()
# swa_model = swa_model.cuda()
"""
loading the best performing weights and calling the test function to display the results
"""


#change the name of the unet as required
new_attention_byol_unet,_,_,_ = load_ckp(best_model_path, new_attention_byol_unet,optimizer)

#change the name of the unet as required
loss , score = test_model(new_attention_byol_unet,test_loader_segmentation_luchi,criterion)

In [ ]:
print(loss)
print(score)

In [ ]:
# class Dataset_for_test(Dataset):
#   def __init__(self,df,transform = None):
#     self.df = df
#     self.transforms = transform

#   def __len__(self):
#     return len(self.df)

#   def __getitem__(self,idx):
#     img_path = self.df['img'][idx]
#     mask_path = self.df['mask'][idx]

#     X = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
#     X = np.expand_dims(X,axis=-1)

#     Y = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
#     Y = np.expand_dims(Y,axis=-1)

#     mask = np.zeros((Y.shape[0],Y.shape[1],1))
#     mask = np.maximum(mask,Y)
#     mask = mask.astype("float32")
    
# #     if transform:
# #         transformed = self.transforms(image=X, mask = mask)
# #         img = transformed['image']
# #         mask = transformed['mask']
    
#     img = cv2.resize(X,(224,224),interpolation=cv2.INTER_NEAREST)
#     mask = cv2.resize(mask,(224,224),interpolation=cv2.INTER_NEAREST)

#     mask = mask/255
#     img = img/255
#     mask = np.expand_dims(mask,axis=-1).transpose((2,0,1))
#     img = np.expand_dims(img,axis=0)

#     img = torch.from_numpy(img)
#     img.unsqueeze(0)
#     mask = torch.from_numpy(mask)

#     return (img,mask)


In [ ]:
# data_test_luchi = Dataset_for_test(test_df)

# # trainset_luchi, valset_luchi = random_split(data_train_luchi, [train_split_luchi,test_split_luchi])
# # trainset_kasthuri,valset_kasthuri = random_split(data_train_kasthuri,[train_split_kasthuri,test_split_kasthuri])

# train_loader = torch.utils.data.DataLoader(dataset=data_test_luchi, batch_size=1)



In [ ]:

# for img,mask in train_loader:
#     img = img.to(device,dtype = torch.float)
#     mask = mask.to(device,dtype = torch.float)   
#     predicted_mask = new_attention_byol_unet.forward(img)
#     print(predicted_mask.shape)
# #     loss = criterion(predicted_mask,mask)
# #     score = iou_batch(predicted_mask,mask)
#     plot_img(img,mask,prediction=predicted_mask)
# #     running_test_loss.append(loss.item())
# #     running_test_score.append(score)
# # return np.mean(running_test_loss),np.mean(running_test_score)

In [ ]:
"""
function to plot the traing and validation loss and iou
"""

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.plot(train_loss,label='train_loss')
plt.plot(val_loss,label='val_loss')
plt.legend()
plt.title('Loss Plot')
plt.subplot(1,2,2)
plt.plot(train_iou,label='train_iou')
plt.plot(val_iou,label='val_iou')
plt.legend()
plt.title('IOU Plot')
plt.show()

In [ ]:
"""
Function to plot the image and its corresponding mask
"""

iter_ = iter(val_loader_segmentation_luchi)
image,mask = next(iter_)
image = image.to(device,dtype=torch.float)
mask = mask.to(device,dtype=torch.float)
y_pred = new_attention_byol_unet.forward(image)


plt.figure(figsize=(20,15))
for i in range(0,3):
    plt.subplot(3,5,i+1)
    plt.title('Actual image')
    plt.imshow(image_convert(image[i]))
for i in range(0,3):
    plt.subplot(3,5,i+5+1)
    plt.title('Actual mask')
    plt.imshow(mask_convert(mask[i]),cmap='gray')
    
    print(np.unique(np.array(mask[i].cpu().squeeze()).flatten()))
for i in range(0,3):
    plt.subplot(3,5,i+10+1)
    plt.title('Predicted mask')
    plt.imshow(mask_convert(y_pred[i]),cmap='gray')
    print(np.unique(np.array(y_pred[i].detach().cpu().squeeze()).flatten()))
plt.show()